# **Лабораторная работа "Стеганография"**

In [ ]:
!pip install -r requirements.txt

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Разбиение изображения на биты

Каждый пиксель изображения содержит три цветовых канала `RGB`, где каждый канал представлен 8-битным числом от 0 до 255. Метод `LSB` использует младшие биты этих каналов для скрытия данных.

Например, если мы шифруем информацию двумя битами, то наш текст будет храниться в последних двух битах оригинального изображения до шифрования.

Возьмем пиксель со значениями:

- Красный канал: 173 (в двоичной системе `10101101`)
- Зеленый канал: 210 (`11010010`)
- Синий канал: 95 (`01011111`)

Если мы хотим спрятать символ `"A"`, который в бинарном виде выглядит как `01000001`, мы разбиваем его на четыре двухбитных фрагмента: `01`, `00`, `00`, `01`. Затем подставляем эти фрагменты вместо двух младших битов в цветовых каналах:

После внедрения получаем:

- Красный: `10101101` → `10101101` (без изменений, так как последние два бита уже `01`)
- Зеленый: `11010010` → `11010000` (последние два бита заменены на `00`)
- Синий: `01011111` → `01011101` (последние два бита заменены на `01`)

Таким образом, мы спрятали весь символ `"A"` всего в одном пикселе, используя по два младших бита из каждого цветового канала.

In [ ]:
# Путь к фотографии
path_image = '../1007.png'

image = cv2.imread(path_image)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(image_rgb)
plt.title("Зашифрованное изображение")
plt.xticks([])
plt.yticks([])
plt.show()

Создаем битовые плоскости изображения:

- Каждый пиксель раскладываем на 8 бит
- Для n-го бита строится бинарная матрица:
- - Если n-й бит = 1 → черный пиксель
- - Если n-й бит = 0 → белый пиксель

Таким образом получаем `3n` монохромных изображений, где каждое показывает распределение конкретного бита по всему изображению. Младшие биты (LSB) обычно выглядят как случайный шум, а старшие содержат основную структурную информацию изображения.

In [ ]:
colors = {0: 'red', 1: 'green', 2: 'blue'}

def rastr(image: np.ndarray, color: int, bit: int) -> np.ndarray:
    ysize, xsize = image.shape[:2]
    image_result = np.zeros((ysize, xsize))

    for i in range(ysize):
        for j in range(xsize):
            image_result[i][j] = image[i][j][color] & bit
    return image_result

In [ ]:
bit_max = 8

plt.figure(figsize=(40,80))
for bit in range(bit_max):
    for color_idx, color_name in colors.items():
        plt.subplot(bit_max, 3, bit * 3 + color_idx + 1)

        plt.title(f"Бит: {bit}\nЦвет: {color_name}")
        plt.imshow(rastr(image_rgb, color_idx, bit), cmap='gray')
        plt.xticks([])
        plt.yticks([])

plt.show()

In [ ]:
color_temp, bit_temp = 1, 2

plt.title(f"Бит: {bit_temp}\nЦвет: {colors.get(color_temp)}")
plt.imshow(rastr(image_rgb, color_temp, bit_temp), cmap='gray')
plt.xticks([])
plt.yticks([])
plt.show()

## **Гистограмма**

In [ ]:
def channel(index: int = 0):
    ysize, xsize = image_rgb.shape[:2]
    hist = np.zeros(256)
    
    for line in image_rgb:
        for pixel in line:
            # Исправлено: pixel[index] уже дает значение 0-255
            hist[pixel[index]] += 1  # убрал -1

    return hist

# Получаем гистограммы
r_hist = channel(0)
g_hist = channel(1)
b_hist = channel(2)

print("Red channel histogram sums:", np.sum(r_hist))

# Построение гистограмм - ПРАВИЛЬНЫЙ способ
plt.figure(figsize=(8, 8))

# Способ 1: Использовать готовые гистограммы как данные
plt.subplot(3, 1, 1)
plt.plot(range(256), r_hist, color='red', alpha=0.7)
plt.title('Красный')
plt.xlim([0, 255])
plt.xlabel('')

plt.subplot(3, 1, 2)
plt.plot(range(256), g_hist, color='green', alpha=0.7)
plt.title('Зеленый')
plt.xlim([0, 255])

plt.subplot(3, 1, 3)
plt.plot(range(256), b_hist, color='blue', alpha=0.7)
plt.title('Синий')
plt.xlim([0, 255])

plt.tight_layout()
plt.show()

## **Серые оттенки**

In [ ]:
def image_to_grey(image):
    ysize, xsize = image_rgb.shape[:2]
    image_grey = np.zeros((ysize, xsize))

    for i in range(ysize):
        for j in range(xsize):
            r, g, b = image[i][j]
            temp_grey = (r + g + b) / 3
            image_grey[i][j] = temp_grey

    return image_grey

image_grey = image_to_grey(image_rgb)

plt.imshow(image_grey, cmap='gray')
plt.show()

## **Дешифровка (LSB)**

Для дешифрования сделаем:

1) Приведем двумерный массив к одномерному;
2) Создадим список для хранения зашифрованных битов по трем каналам;
3) Полученный массив разделяем на 8 битов и приводим к текстовому формату (например по кодеровке UTF-8).

In [ ]:
def lsb(image: np.ndarray, bit: int) -> str:
    line = image.reshape(-1, 3)

    bit_list = []
    for pixel in line:
        r, g, b = pixel
        for chanel in (r, g, b):
            bit_list.extend([(chanel >> i) & 1 for i in range(bit-1, -1, -1)])

    bit_data = bytearray()
    for i in range(0, len(bit_list), 8):
        if i + 8 > len(bit_list):
            break
        val = 0
        for j in range(8):
            val = (val << 1) | bit_list[i + j]
        bit_data.append(val)

    try:
        text = bit_data.decode('utf-8', errors='replace')
        return "Текст (UTF-8):", text.split('\x00')[0]  
    except:
        text = bit_data.decode('latin1', errors='replace')
        return "Текст (Latin-1):", text.split('\x00')[0]



with open('test.txt', 'w') as file:
    file.writelines(lsb(image_rgb, 2))